In [1]:
### 前回からの修正点
# ・変数の修正（一度使った名前は使わない）
# ・データの読み込み方法（データシートに記載されているファイル名を使用（Pandas））
# ・データシート修正（'No'，'Modifiled_info'追加）
# ・フォルダ構成修正
# ・パラメータ入力位置を全部上部のセルにもってくる
# ・パラメータ自動記入
# 

In [2]:
import os
import csv
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from scipy import signal
import cv2
from sklearn.metrics import r2_score
import pandas as pd

In [3]:
##### データフォルダ・データシート作成（データフォルダが存在しない場合のみ実行）

data_folder = os.getcwd().replace(os.sep, "/") + "/Data"
datasheet_ver = 3
datasheet_path = data_folder + "/datasheet_ver" + str(datasheet_ver) + ".csv"
print("data_folder: " + data_folder)
print("datasheet: " + datasheet_path)

cols = ["No", "Gonio", "Echo", "Date", "Subject", "Pattern", "Additional Info", "Modified_Info", "Trial Num", "Depth", "Gain", "Focus", "Frequency", "Dynamic Range", 
        "Max Corners", "Quality Level", "Min Distance", "Block Size", "Trim Width", "Trim Height", "RMSE", "R2", "Corrcoef", "Updated Date"]

# データフォルダが存在しなければ作成
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    print("Data folder was successfully created.")
else:
    print("Data folder already exists.")

# データシートが存在しなければ作成
if not os.path.exists(datasheet_path):
    df = pd.DataFrame(columns=cols)
    print(df)
    df.to_csv(datasheet_path, encoding="shift_jis", index=False)
    print("Datasheet was successfully created.")
        
else:
    print("Datasheet already exists.")

data_folder: C:/Users/katagi/Desktop/Research/UltrasoundImaging/Data
datasheet: C:/Users/katagi/Desktop/Research/UltrasoundImaging/Data/datasheet_ver3.csv
Data folder already exists.
Datasheet already exists.


In [4]:
##### ターゲットのデータがデータシートに既に記入されているか（一度同じ条件で解析されているか）を確認

data_sheet = pd.read_csv(datasheet_path, header=0, encoding='shift_jis')
print(data_sheet)

## INFO
# 実験日
date = "20221021"

# 被験者
subject = "Katagi"

# 実験パターン
pattern = 2

# 補足
additional_info = ""

# 修正情報
modified_info = ""

# # 試行No.
# trial_num = 2

# 試行数
num_of_trial = 5


## SOFT PARAM
# 深度(mm)
depth = 20

# ゲイン(%)
gain = 50

# フォーカス
focus = "0_7"

# 周波数(MHz)
frequency = 7

# ダイナミックレンジ(dB)
dynamic_range = 36


## OPTICALFLOW PARAM
maxCorners=500       # 特徴点の最大数
qualityLevel=0.15     # 特徴点を選択するしきい値で、高いほど特徴点は厳選されて減る。
minDistance=15       # 特徴点間の最小距離
blockSize=15         # 特徴点の計算に使うブロック（周辺領域）サイズ

## トリミング範囲
trim_w = 80
trim_h = 150



# ## ターゲットのデータがデータシートに存在するか確認
# # target_data_block = data_sheet[(data_sheet['Date']==int(date)) & (data_sheet['Subject']==subject) & (data_sheet['Pattern']==pattern) \
# #                               & (data_sheet['Additional Info']==additional_info) & (data_sheet['Modified Info']==modified_info) \
# #                               & (data_sheet['Depth']==depth) & (data_sheet['Gain']==gain) & (data_sheet['Focus']==focus) \
# #                               & (data_sheet['Frequency']==frequency) & (data_sheet['Dynamic Range']==dynamic_range)]
# target_data_block = data_sheet[(data_sheet['Date']==int(date)) & (data_sheet['Subject']==subject) & (data_sheet['Pattern']==pattern) & (data_sheet['Additional Info']==additional_info)]

# print("target_data_block: " + str(target_data_block))

Empty DataFrame
Columns: [No, Gonio, Echo, Date, Subject, Pattern, Additional Info, Modified_Info, Trial Num, Depth, Gain, Focus, Frequency, Dynamic Range, Max Corners, Quality Level, Min Distance, Block Size, Trim Width, Trim Height, RMSE, R2, Corrcoef, Updated Date]
Index: []

[0 rows x 24 columns]


In [ ]:
##### パラメータ


### ゴニオデータ処理用 パラメータ

gonio_frame_rate = 100
echo_frame_rate = depth_param[depth][0]
total_time = 44
numOfData = echo_frame_rate * total_time
interpolate_rate = gonio_frame_rate * echo_frame_rate

# csvファイル内の目的データの開始位置
cut_time = 12
start_row = gonio_frame_rate * cut_time
end_row = start_row + gonio_frame_rate * total_time
target_column = 1

# ローパスフィルタ パラメータ
fp = 1.5 # 通過域端周波数[Hz] #フーリエ変換にあわせて調整
fs = 7.5 # 阻止域端周波数[Hz] # fp*5くらい
gpass = 3 # 通過域端最大損失[dB]
gstop = 40 # 阻止域端最小損失[dB]

time = np.linspace(0, total_time, numOfData)

print("echo_frame_rate: " + str(echo_frame_rate))
print("numOfData: " + str(numOfData))
print("interpolate_rate: " + str(interpolate_rate))
print("start_row: " + str(start_row))
print("end_row: " + str(end_row))
print("time: ")
print(time, time.shape)



### エコーデータ処理用 パラメータ

# 計測時間(s)
start_time = 12
end_time = 56

# トリミング（元動画の超音波画像の部分のみに切り取る）
# 元動画サイズ：(1172, 608)
top, bottom = 40, 600
left, right = depth_param[depth][1][0], depth_param[depth][1][1]
# # トリミング後サイズ：(right - left, 560)
print("left, right: " + str(left) + ", " + str(right))


# 特徴点数
feature_num = 50


# フレーム内へ角度情報を挿入する位置
wrist_angle_position = (40, 530)

### 解析用 パラメータ

# グラフ
min_height = -80
max_height = 30

# 正則化パラメータ
# lam = 0.1
lam = 1.0

In [ ]:
### ターゲットのデータがなければデータシートに記入
# print(target_data_block.empty)
# if not target_data_block.empty:



### Arrange Target Data Folder

# 新しい解析フォルダを作成し，その中に必要なフォルダを一通り作成する
# 同じ実験データで条件を変更して解析し直すとき用

# if modified_info == "":
#     analysis_path = data_path + "/Analysis1"
# else:
#     analysis_path = data_path + "/" + modified_info


# if os.path.exists(analysis_path):
#     print(analysis_path + " folder already exists.")
# else:
#     # EchoData フォルダ直下に新しいフォルダを作成
#     processed_movie_path = data_path + "/EchoData/ProcessedMovie"
#     if not os.path.exists(processed_movie_path):
#         os.makedirs(processed_movie_path)
    
#     # analysis_path フォルダを作成
#     os.makedirs(analysis_path)
    
#     # new_analysis フォルダ直下に新しいフォルダを作成
#     processed_data_path = analysis_path + "/ProcessedData"
#     results_path = analysis_path + "/Results"
#     os.makedirs(processed_data_path)
#     os.makedirs(results_path)
    
#     # ProcessedData フォルダ直下に新しいフォルダを作成
#     echo_processed_path = processed_data_path + "/EchoProcessed"
#     gonio_processed_path = processed_data_path + "/GonioProcessed"
#     os.makedirs(echo_processed_path)
#     os.makedirs(gonio_processed_path)
    
#     # EchoProcessed フォルダ直下に新しいフォルダを作成
#     opticalflow_movie_path = echo_processed_path + "/OpticalflowMovie"
#     original_fp_path = echo_processed_path + "/OriginalFP"
#     processed_fp_path = echo_processed_path + "/ProcessedFP"
#     echo_comparison_path = echo_processed_path + "/EchoComparison"
#     os.makedirs(opticalflow_movie_path)
#     os.makedirs(original_fp_path)
#     os.makedirs(processed_fp_path)
#     os.makedirs(echo_comparison_path)
    
#     # GonioProcessed フォルダ直下に新しいフォルダを作成
#     processed_theta_path = gonio_processed_path + "/ProcessedTheta"
#     gonio_comparison_path = gonio_processed_path + "/GonioComparison"
#     os.makedirs(processed_theta_path)
#     os.makedirs(gonio_comparison_path)
    
#     # Results フォルダ直下に新しいフォルダ・結果用csvファイルを作成
#     point_behavior_path = results_path + "/PointBehavior"
#     angle_estimation_path = results_path + "/AngleEstimation"
#     each_datasheet_path = results_path + "/each_datasheet.csv"
#     os.makedirs(point_behavior_path)
#     os.makedirs(angle_estimation_path)
#     # 各解析ごとの結果用csvファイル作成
#     with open(each_datasheet_path, "w", newline="") as f:
#         writer = csv.writer(f)
#         writer.writerow(["", "INFO", "", "", "", "", "", "", "SOFT PARAM", "", "", "", "", "", "PATH", "", "", "RESULTS"])
#         writer.writerow(["No", "Date", "Subject", "Patern", "Additional Info", "Modified_Info", "Trial Num", "", "Depth", "Gain", "Focus", "Frequency", "Dynamic Range", "", "Gonio", "Echo", "", "RMSE", "R2", "Corrcoef"])

#     print(processed_movie_path + " folder was successfully created.")
#     print(new_analysis + " folder was successfully created.")
#     print(processed_data_path + " folder was successfully created.")
#     print(results_path + " folder was successfully created.")
#     print(echo_processed_path + " folder was successfully created.")
#     print(gonio_processed_path + " folder was successfully created.")
#     print(opticalflow_movie_path + " folder was successfully created.")
#     print(original_fp_path + " folder was successfully created.")
#     print(processed_fp_path + " folder was successfully created.")
#     print(echo_comparison_path + " folder was successfully created.")
#     print(processed_theta_path + " folder was successfully created.")
#     print(gonio_comparison_path + " folder was successfully created.")
#     print(point_behavior_path + " folder was successfully created.")
#     print(angle_estimation_path + " folder was successfully created.")
#     print(each_datasheet + " folder was successfully created.")

In [ ]:
#### ターゲットデータ取得（1個）

# データナンバー
##この値を変えると自動的に、実験日、被験者、実験パターン、試行No.を抽出する
# ※ 解析対象を変更するときは、この番号だけ変えればよい
data_No = 19

# target_data = data_sheet[ data_sheet['No']==data_No ]
target_data = data_sheet.loc[data_sheet.No == data_No].copy()
print('--target_data-----------')
print(target_data)
print('------------------------')


print('--target_data information--')
## INFO
# 実験日
date = str(int(target_data["Date"].iloc[0]))
print("date            : " + date)

# 被験者
subject = str(target_data["Subject"].iloc[0])
print("subject         : " + subject)

# 実験パターン
pattern = int(target_data['Pattern'].iloc[0])
print("pattern         : " + str(pattern))

# 補足
additional_info = str(target_data["Additional Info"].iloc[0])
if additional_info == "nan":
    additional_info = ""
print("additional_info : " + additional_info)

# 修正情報
modified_info = str(target_data["Modified Info"].iloc[0])
if modified_info == "nan":
    modified_info = ""
print("modified_info   : " + modified_info)

# 試行No.
trial_num = int(target_data["Trial Num"].iloc[0])
print("trial_num       : " + str(trial_num))


## SOFT PARAM
# 深度(mm)
depth = int(target_data["Depth"].iloc[0])
print("depth           : " + str(depth))

# ゲイン(%)
gain = int(target_data["Gain"].iloc[0])
print("gain            : " + str(gain))

# フォーカス
focus = str(target_data["Focus"].iloc[0])
print("focus           : " + str(focus))

# 周波数(MHz)
frequency = int(target_data["Frequency"].iloc[0])
print("frequency       : " + str(frequency))

# ダイナミックレンジ(dB)
dynamic_range = int(target_data["Dynamic Range"].iloc[0])
print("dynamic_range   : " + str(dynamic_range))


## PATH
# ゴニオ（関節角度）データファイル名
gonio_file = str(target_data["Gonio"].iloc[0])
print("gonio_file      : " + str(gonio_file))

# エコー動画ファイル名
echo_file = str(target_data["Echo"].iloc[0])
print("echo_file       : " + str(echo_file))



print('---------------------------')


# 深度によってフレームレートとトリミングする範囲が変わる
depth_param = {20 : [73, [70, 1100]], 30 : [62, [85, 1085]], 40 : [54, [200, 970]], 50 : [48, [280, 890]], 60 : [43, [325, 845]]}
# Depth:              20          30         40         50         60
# Frame Rate:         73          62         54         48         43
# Triming Range:  [70, 1100] [85, 1085] [200, 970] [280, 890] [325, 845]


current_directory = os.getcwd().replace(os.sep, "/")
print("current_directory: " + current_directory)

data_path = current_directory + "/Data/" + date + "_" + subject + "_" + additional_info
print("data_path: " + data_path)

In [ ]:
##### 解析パス設定

if modified_info == "":
    analysis_path = data_path + "/Analysis1"
else:
    analysis_path = data_path + "/" + modified_info